In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Load the data, and separate the target
iowa_file_path = 'train.csv'
home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice

# Create X (After completing the exercise, you can return to modify this line!)
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']

# Select columns corresponding to features, and preview the data
X = home_data[features]
X.head()

# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

# Define a random forest model
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 21,857


In [2]:
rf_model_on_full_data = RandomForestRegressor(random_state=1)
rf_model_on_full_data.fit(X,y)

RandomForestRegressor(random_state=1)

In [3]:
test_data_path = 'test.csv'
test_data = pd.read_csv(test_data_path)
# print(test_data.columns)
test_X = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
C = test_data[test_X]
# print("C",X)
test_preds = rf_model_on_full_data.predict(C)

In [4]:
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)

In [5]:
from fastapi import FastAPI
from uvicorn import Config,Server
from pydantic import BaseModel
from sklearn.datasets import load_iris

In [6]:
import joblib
joblib.dump(rf_model_on_full_data, "./random_forest.joblib")

['./random_forest.joblib']

In [7]:
loaded_rf = joblib.load("./random_forest.joblib")

In [8]:
# loaded_rf.predict(X)
# home_data.dtypes
print(X)

      LotArea  YearBuilt  1stFlrSF  2ndFlrSF  FullBath  BedroomAbvGr  \
0        8450       2003       856       854         2             3   
1        9600       1976      1262         0         2             3   
2       11250       2001       920       866         2             3   
3        9550       1915       961       756         1             3   
4       14260       2000      1145      1053         2             4   
...       ...        ...       ...       ...       ...           ...   
1455     7917       1999       953       694         2             3   
1456    13175       1978      2073         0         2             3   
1457     9042       1941      1188      1152         2             4   
1458     9717       1950      1078         0         1             2   
1459     9937       1965      1256         0         1             3   

      TotRmsAbvGrd  
0                8  
1                6  
2                6  
3                7  
4                9  
...      

In [9]:
import asyncio
import numpy as np


app = FastAPI()


class UserInput(BaseModel):
    LotArea: int
    YearBuilt: int
    stFlrSF: int
    ndFlrSF: int
    FullBath: int
    BedroomAbvGr: int
    TotRmsAbvGrd: int



@app.get('/')
async def index():


    return {"test"}


config = Config(app)


server = Server(config=config)


loop = asyncio.get_event_loop()


loop.create_task(server.serve())


@app.post('/predict')
async def predict(user_input: UserInput):
    input_array = np.array(
        [[user_input.LotArea, user_input.YearBuilt, user_input.stFlrSF, user_input.ndFlrSF, user_input.FullBath, user_input.BedroomAbvGr, user_input.TotRmsAbvGrd]])
    prediction = loaded_rf.predict(input_array)
    print(prediction)
    return {"predicted_price": float(prediction[0])}

INFO:     Started server process [4084]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


[194653.69666667]
INFO:     127.0.0.1:5582 - "POST /predict HTTP/1.1" 200 OK


c:\Users\DELL\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
